In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
import numpy as np
import os
from sklearn.model_selection import train_test_split
from qiskit_aer import AerSimulator
from PIL import Image  # Ajout de l'import manquant pour Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau  # Callbacks pour améliorer l'entraînement
import logging  # Pour le logging


In [2]:

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# 1️⃣ Chargement du dataset RGB
def load_dataset(dataset_path):
    data, labels = [], []
    classes = {'fire': 1, 'non_fire': 0}
    
    for class_name, label in classes.items():
        folder = os.path.join(dataset_path, class_name)
        if not os.path.exists(folder):  # Vérification que le dossier existe
            logger.error(f"Le dossier {folder} n'existe pas.")
            raise FileNotFoundError(f"Le dossier {folder} n'existe pas.")
        
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            try:
                img = Image.open(img_path).convert("RGB").resize((128, 128))
                img_array = np.array(img) / 255.0
                data.append(img_array)
                labels.append(label)
            except Exception as e:
                logger.warning(f"Erreur lors du chargement de l'image {img_path}: {e}")
    
    return np.array(data), np.array(labels)

# Chargement du dataset
X, y = load_dataset("fire_dataset")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# 2️⃣ Définition de la couche quantique
class QuantumLayer(Layer):
    def __init__(self, num_qubits=4, layers=1, shots=1024, backend=None, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.num_qubits = num_qubits
        self.layers = layers
        self.shots = shots
        self.backend = backend if backend else AerSimulator()

        # Définition des paramètres entraînables du circuit quantique
        self.num_params = num_qubits * layers * 2  # 2 paramètres par qubit (Rx, Rz)
        self.theta = self.add_weight(
            shape=(self.num_params,), 
            initializer="random_uniform", 
            trainable=True
        )

    def build_circuit(self, inputs):
        """Construit un circuit quantique avec des paramètres entraînables."""
        circuit = QuantumCircuit(self.num_qubits)
        params = ParameterVector("θ", self.num_params)

        param_idx = 0

        # 🔹 Encodage des features en Ry
        for i in range(self.num_qubits):
            circuit.ry(inputs[i] * np.pi, i)

        # 🔹 Application du PQC
        for _ in range(self.layers):
            for qubit in range(self.num_qubits):
                circuit.rx(params[param_idx], qubit)
                circuit.rz(params[param_idx + 1], qubit)
                param_idx += 2
            
            # 🔹 CNOT en chaîne
            for qubit in range(self.num_qubits - 1):
                circuit.cx(qubit, qubit + 1)
        
        return circuit, params

    def run_quantum_circuit(self, inputs_np):
        """Fonction exécutée en mode NumPy via tf.numpy_function."""
        batch_size = inputs_np.shape[0]
        results = []

        for i in range(batch_size):
            try:
                # Construire le circuit quantique
                circuit, params = self.build_circuit(inputs_np[i])
                bound_circuit = circuit.assign_parameters({params[j]: self.theta.numpy()[j] for j in range(self.num_params)})
                transpiled_circuit = transpile(bound_circuit, self.backend)
                
                # Exécuter le circuit
                job = self.backend.run(transpiled_circuit, shots=self.shots)
                counts = job.result().get_counts()
                
                # Convertir les counts en un vecteur de features
                feature_vector = [counts.get(bin(j)[2:].zfill(self.num_qubits), 0) / self.shots for j in range(2**self.num_qubits)]
                results.append(feature_vector)
            except Exception as e:
                logger.error(f"Erreur lors de l'exécution du circuit quantique : {e}")
                results.append(np.zeros(2**self.num_qubits))  # Retourner un vecteur nul en cas d'erreur
        
        return np.array(results, dtype=np.float32)

    def call(self, inputs):
        """Utilisation de tf.numpy_function pour éviter l'erreur avec KerasTensors."""
        outputs = tf.numpy_function(self.run_quantum_circuit, [inputs], tf.float32)
        outputs.set_shape((None, 2**self.num_qubits))  # Définir la bonne forme
        return outputs

In [5]:
# 3️⃣ Définition du modèle hybride
def build_hybrid_model():
    inputs = Input(shape=(128, 128, 3))

    # 🔹 Partie CNN
    x = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    features = Dense(4, activation='relu', name="features")(x)  # 🔹 4 features pour correspondre à 4 qubits

    # 🔹 Partie quantique
    quantum_output = QuantumLayer(num_qubits=4, layers=1)(features)

    # 🔹 Classification finale
    x = Dense(16, activation="relu")(quantum_output)
    x = Dense(1, activation="sigmoid")(x)  # 🔹 Classification binaire

    model = Model(inputs=inputs, outputs=x)
    return model



In [6]:
# 4️⃣ Création du modèle hybride
hybrid_model = build_hybrid_model()

In [7]:
# 5️⃣ Compilation du modèle
hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [8]:

# 6️⃣ Callbacks pour améliorer l'entraînement
callbacks = [
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1)
]

In [9]:

# 7️⃣ Affichage du résumé du modèle
hybrid_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ features (Dense)                │ (None, 4)              │       262,148 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ quantum_layer (QuantumLayer)    │ (None, 16)             │             8 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 281,837 (1.08 MB)

 Trainable params: 281,837 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 8️⃣ Entraînement du modèle
logger.info("Début de l'entraînement du modèle...")
history = hybrid_model.fit(
    X_train, y_train, 
    epochs=5,  # Augmentation du nombre d'epochs
    batch_size=64, 
    validation_data=(X_test, y_test), 
    callbacks=callbacks,  # Ajout des callbacks
    verbose=1
)

INFO:__main__:Début de l'entraînement du modèle...


Epoch 1/5


d:\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['conv2d/kernel', 'conv2d/bias', 'conv2d_1/kernel', 'conv2d_1/bias', 'features/kernel', 'features/bias', 'quantum_layer/variable'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
INFO:qiskit.passmanager.base_tasks:Pass: ContainsInstruction - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 2.99549 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 5.61953 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.99587 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Size - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)

 1/20 ━━━━━━━━━━━━━━━━━━━━ 5:03 16s/step - accuracy: 0.3750 - loss: 0.6931

INFO:qiskit.passmanager.base_tasks:Pass: ContainsInstruction - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Size - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Optimize1qGatesDecomposition - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: GatesInBasis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.

...

14/20 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.4575 - loss: 0.6932

INFO:qiskit.passmanager.base_tasks:Pass: ContainsInstruction - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 0.99897 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Size - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Optimize1qGatesDecomposition - 1.50681 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: GatesInBasis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.

15/20 ━━━━━━━━━━━━━━━━━━━━ 24s 5s/step - accuracy: 0.4597 - loss: 0.6932

INFO:qiskit.passmanager.base_tasks:Pass: ContainsInstruction - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: UnitarySynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: HighLevelSynthesis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: BasisTranslator - 1.00660 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Size - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Optimize1qGatesDecomposition - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: InverseCancellation - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: GatesInBasis - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: Depth - 0.00000 (ms)
INFO:qiskit.passmanager.base_tasks:Pass: FixedPoint - 0.00000 (ms)
INFO:qiskit.

16/20 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step - accuracy: 0.4617 - loss: 0.6932

In [ ]:
# 9️⃣ Évaluation sur le test set
logger.info("Évaluation du modèle sur le test set...")
test_loss, test_acc = hybrid_model.evaluate(X_test, y_test)
print(f"🎯 Précision finale du modèle hybride : {test_acc * 100:.2f}%")

In [ ]:
from tensorflow.keras.preprocessing import image

# Fonction pour charger et prétraiter une image
def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB").resize((128, 128))
    img_array = np.array(img) / 255.0  # Normalisation
    return np.expand_dims(img_array, axis=0)  # Ajouter la dimension du batch

# Charger l'image
img_path = "foret.jpg"
img_processed = load_and_preprocess_image(img_path)

# Faire une prédiction
prediction = hybrid_model.predict(img_processed)

# Afficher le résultat
if prediction[0] > 0.5:
    print("🔥 L'image est classée comme feu.")
else:
    print("🌲 L'image est classée comme non-feu.")

#################################################################################
#################################################################################
#################################################################################
#################################################################################
#################################################################################


## Second one